In [149]:
import numpy as np
import torch
from deepcase_copy.context_builder import ContextBuilder
from deepcase_copy.preprocessing import Preprocessor

builder = ContextBuilder.load('save/builder.save')
preprocessor = Preprocessor(
    length=10,  # 10 events in context
    timeout=86400,  # Ignore events older than 1 day (60*60*24 = 86400 seconds)
)
context, events, labels, mapping = preprocessor.csv('alerts.csv', verbose=True)

# In case no labels are provided, set labels to -1
# IMPORTANT: If no labels are provided, make sure to manually set the labels
# before calling the interpreter.score_clusters method. Otherwise, this will
# raise an exception, because scores == NO_SCORE cannot be computed.
if labels is None:
    labels = np.full(events.shape[0], -1, dtype=int)

# Cast to cuda if available
if torch.cuda.is_available():
    builder = builder.to('cuda')
    events = events.to('cuda')
    context = context.to('cuda')


# Split into train and test sets (20:80) by time - assuming events are ordered chronologically
events_train  = events [:events.shape[0]//5 ]
events_test   = events [ events.shape[0]//5:]

context_train = context[:events.shape[0]//5 ]
context_test  = context[ events.shape[0]//5:]

labels_train  = labels [:events.shape[0]//5 ]
labels_test   = labels [ events.shape[0]//5:]

Loading: 100%|██████████| 3353/3353 [00:00<00:00, 4625.17it/s]


In [150]:
import torch

def get_unique_indices_per_row(tensor):
    indices_list = []
    row_list = []
    indices_list_set = set()
    
    # Iterate over each row
    for row in range(len(tensor)):
        curr = tuple(tensor[row].tolist())
        
        # Check if the current row already exists in row_list using torch.equal
        if curr in indices_list_set:
            continue
        
        # Append unique row and its index
        row_list.append(curr)
        indices_list.append(row)
        indices_list_set.add(curr)
    
    # Return the indices and unique rows
    return indices_list, row_list
indices, rows = get_unique_indices_per_row(context_test)

In [151]:
context_filtered = torch.tensor(rows)
events_filtered = events_test[indices].clone().detach()
labels_filtered = labels_test[indices].clone().detach()

if torch.cuda.is_available():
    context_filtered = context_filtered.to('cuda')
    events_filtered = events_filtered.to('cuda')
    labels_filtered = labels_filtered.to('cuda')

In [176]:
l = 100
chosen_index = 0
context_picked = context_filtered[chosen_index:chosen_index+l].detach()
events_picked = events_filtered[chosen_index:chosen_index+l].detach()
labels_picked = labels_filtered[chosen_index:chosen_index+l]

In [180]:
from deepcase_copy.context_builder.loss import LabelSmoothing

max_iterations_global = 10

def max_to_one(tensor):
    max_indices = torch.argmax(tensor, dim=-1, keepdim=True)
    result = torch.zeros_like(tensor)
    result.scatter_(-1, max_indices, 1.0)
    return result

def get_results(results):
    results_picked = torch.topk(results[0][0][0], 3)
    exp = results_picked.values.exp()
    res_indices = results_picked.indices
    s = ""
    for j in range(3):
        s += f"[{res_indices[j].item()}] {round(exp[j].item(), 3)}, "
    return res_indices, s

def compute_change(trace, original, epsilon=0.1):
    a = torch.clamp(original - epsilon, min=0)
    b = (trace >= a).float() * trace + (trace < a).float() * a
    c = (b > original + epsilon).float() * (original + epsilon) + (b <= original + epsilon).float() * b
    return max_to_one(c)

def bim_attack(context_given, target_given, alpha=0.1, epsilon=0.1, num_iterations=max_iterations_global):
    change = None
    original_context = builder.embedding_one_hot(context_given)
    context_processed = builder.embedding_one_hot(context_given)
    criterion = LabelSmoothing(builder.decoder_event.out.out_features, 0.1)    
    changes = []
    for i in range(num_iterations):
        context_processed.requires_grad_(True)
        output = builder.predict(context_processed)
        indices_of_results, prediction_str = get_results(output)
        changes.append({
            "changed_to": torch.argmax(context_processed, axis=-1).tolist()[0],
            "prediction_str": prediction_str,
        })
        if target_given[0] != indices_of_results[0]:
            break
        loss = criterion(output[0][0], target_given)
        context_processed.retain_grad()
        loss.backward(retain_graph=True)
        grad = context_processed.grad.sign()
        if change is None:
            change = alpha * grad
        else:
            change += alpha * grad
        context_processed = context_processed + change
        context_processed = compute_change(context_processed, original_context, epsilon)
    return changes

current_trace_num = 0
incorrect=0
changed=0
timeout=0
for con, e in zip(context_picked, events_picked.unsqueeze(1)):
    con.resize_(1, con.size()[-1])
    changes_needed = bim_attack(context_given=con, target_given=e, alpha=0.1, epsilon=0.5, num_iterations=max_iterations_global)
    mode = ""
    if len(changes_needed) == 1:
        incorrect+=1
        mode = "Incorrect"
    elif len(changes_needed) == max_iterations_global:
        timeout+=1
        mode = "Timeout"
    else:
        changed += 1
        mode = f"Changed {len(changes_needed)}"
    print(f"{current_trace_num}: {con[0].tolist()} == {e.tolist()} {mode}")
    for change in changes_needed:
        print(f"{" "*(len(str(current_trace_num)) + 2)}{change["changed_to"]} -> {change['prediction_str']}")
    print()
    current_trace_num += 1
print(f"{incorrect=} {changed=} {timeout=}")

0: [71, 71, 71, 71, 71, 71, 71, 71, 71, 71] == [71] Changed 6
   [71, 71, 71, 71, 71, 71, 71, 71, 71, 71] -> [71] 0.821, [64] 0.047, [72] 0.02, 
   [71, 71, 71, 71, 71, 71, 71, 71, 71, 71] -> [71] 0.821, [64] 0.047, [72] 0.02, 
   [71, 71, 71, 71, 71, 71, 71, 71, 71, 71] -> [71] 0.821, [64] 0.047, [72] 0.02, 
   [71, 71, 71, 71, 71, 71, 71, 71, 71, 71] -> [71] 0.821, [64] 0.047, [72] 0.02, 
   [71, 71, 71, 71, 71, 71, 71, 71, 71, 71] -> [71] 0.821, [64] 0.047, [72] 0.02, 
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -> [0] 0.812, [72] 0.033, [2] 0.015, 

1: [66, 86, 87, 86, 87, 66, 66, 42, 42, 42] == [72] Incorrect
   [66, 86, 87, 86, 87, 66, 66, 42, 42, 42] -> [42] 0.643, [72] 0.11, [78] 0.035, 

2: [72, 72, 72, 72, 72, 85, 72, 72, 72, 72] == [72] Changed 6
   [72, 72, 72, 72, 72, 85, 72, 72, 72, 72] -> [72] 0.826, [64] 0.023, [44] 0.016, 
   [72, 72, 72, 72, 72, 85, 72, 72, 72, 72] -> [72] 0.826, [64] 0.023, [44] 0.016, 
   [72, 72, 72, 72, 72, 85, 72, 72, 72, 72] -> [72] 0.825, [64] 0.023, [44